In [1]:
#importing required libraries
import os
import shutil
import time

import folium
import imageio
import webbrowser
import zipfile
import json
import fileinput

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import path
from branca.colormap import linear
from selenium import webdriver
from PIL import Image
from pathlib import Path

from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
#reading 2015 data into respective dataframes
df1 = pd.read_csv('../Data/2015/Divvy_Trips_2015-Q1.csv')
df2 = pd.read_csv('../Data/2015/Divvy_Trips_2015-Q2.csv')
df3 = pd.read_csv('../Data/2015/Divvy_Trips_2015_07.csv')
df4 = pd.read_csv('../Data/2015/Divvy_Trips_2015_08.csv')
df5 = pd.read_csv('../Data/2015/Divvy_Trips_2015_09.csv')
df6 = pd.read_csv('../Data/2015/Divvy_Trips_2015_Q4.csv')

In [3]:
#Concatenating the dataframes into a single dataframe
frames = [df1,df2,df3,df4,df5,df6]
dfbike = pd.concat(frames)
dfbike.reset_index(drop=True,inplace=True)
dfbike.head(5)

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,4738454,3/31/2015 23:58,4/1/2015 0:03,1095,299,117,Wilton Ave & Belmont Ave,300,Broadway & Barry Ave,Subscriber,Male,1994.0
1,4738450,3/31/2015 23:59,4/1/2015 0:15,537,940,43,Michigan Ave & Washington St,85,Michigan Ave & Oak St,Customer,NaN,NaN
2,4738449,3/31/2015 23:59,4/1/2015 0:11,2350,751,162,Damen Ave & Wellington Ave,67,Sheffield Ave & Fullerton Ave,Subscriber,Male,1992.0
3,4738448,3/31/2015 23:59,4/1/2015 0:19,938,1240,51,Clark St & Randolph St,59,Wabash Ave & Roosevelt Rd,Customer,NaN,NaN
4,4738445,3/31/2015 23:54,4/1/2015 0:15,379,1292,134,Peoria St & Jackson Blvd,216,California Ave & Division St,Subscriber,Male,1976.0


In [4]:
dfbike.columns

Index(['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear'],
      dtype='object')

### Saving all the station names in a list "allSt15"

#### Comparing with 2014 data and only adding the stations which are new into a list "added15"
#### Also, checking for the stations that have been in 2014 but not in this year's data and adding it to "removed14"

In [5]:
allSt15 = list(dfbike['from_station_name'].unique())
len(allSt15)

491

In [6]:
allSt14 = ['Lincoln Ave & Belmont Ave',
 'Halsted St & Maxwell St',
 'Sheffield Ave & Webster Ave',
 'Peoria St & Jackson Blvd',
 'Loomis St & Lexington St',
 'Halsted St & Diversey Pkwy',
 'Canal St & Madison St',
 'Wabash Ave & Grand Ave',
 'Larrabee St & Armitage Ave',
 'Ravenswood Ave & Montrose Ave',
 'State St & 19th St',
 'Canal St & Harrison St',
 'Orleans St & Merchandise Mart Plaza',
 'Lake Park Ave & 47th St',
 'Museum Campus',
 'Clark St & Congress Pkwy',
 'Ashland Ave & Division St',
 'Aberdeen St & Madison St',
 'Franklin St & Chicago Ave',
 'Racine Ave & Belmont Ave',
 'LaSalle St & Illinois St',
 'Wells St & Concord Ln',
 'Michigan Ave & Congress Pkwy',
 'McClurg Ct & Illinois St',
 'Michigan Ave & Pearson St',
 'Mies van der Rohe Way & Chicago Ave',
 'Ashland Ave & Augusta Blvd',
 'Theater on the Lake',
 'Fairbanks Ct & Grand Ave',
 'Franklin St & Arcade Pl',
 'Sheridan Rd & Irving Park Rd',
 'Wells St & Evergreen Ave',
 'Sheffield Ave & Kingsbury St',
 'Halsted St & 18th St',
 'Wentworth Ave & Archer Ave',
 'Michigan Ave & Jackson Blvd',
 'Michigan Ave & Washington St',
 'Clark St & Lincoln Ave',
 'Southport Ave & Roscoe St',
 'Sedgwick St & Webster Ave',
 'Kingsbury St & Erie St',
 'Stetson Ave & South Water St',
 'Larrabee St & Kingsbury St',
 'State St & Harrison St',
 'Sheffield Ave & Fullerton Ave',
 'Stockton Dr & Wrightwood Ave',
 'Damen Ave & Charleston St',
 'May St & Randolph St',
 'Michigan Ave & Lake St',
 'Millennium Park',
 'Morgan St & Lake St',
 'Franklin St & Jackson Blvd',
 'State St & Pearson St',
 'Morgan St & Polk St',
 'Clark St & Wrightwood Ave',
 'Western Ave & Winnebago Ave',
 'State St & Randolph St',
 'LaSalle St & Washington St',
 'Clinton St & Tilden St',
 'Wabash Ave & Cermak Rd',
 'Loomis St & Taylor St',
 'Desplaines St & Kinzie St',
 'Halsted St & James M Rochford St',
 'Clark St & Winnemac Ave',
 'Sheffield Ave & Wellington Ave',
 'Dearborn St & Monroe St',
 'Bissell St & Armitage Ave',
 'St Clair St & Erie St',
 'Lincoln Ave & Leavitt St',
 'LaSalle St & Jackson Blvd',
 'Broadway & Berwyn Ave',
 'Clark St & Wellington Ave',
 'Clark St & Armitage Ave',
 'Broadway & Belmont Ave',
 'Wells St & Hubbard St',
 'Lake Shore Dr & Ohio St',
 'Broadway & Barry Ave',
 'Daley Center Plaza',
 'Wolcott Ave & Polk St',
 'Wabash Ave & Roosevelt Rd',
 'Wentworth Ave & 24th St',
 'Wabash Ave & 8th St',
 'Clark St & North Ave',
 'Halsted St & Wrightwood Ave',
 'May St & Taylor St',
 'Wood St & Division St',
 'Clinton St & Washington Blvd',
 'Halsted St & Blackhawk St',
 'Broadway & Sheridan Rd',
 'Halsted St & Madison St',
 'Michigan Ave & Madison St',
 'Southport Ave & Wellington Ave',
 'Clark St & Waveland Ave',
 'Clark St & Chicago Ave',
 'Columbus Dr & Randolph St',
 'Dearborn St & Adams St',
 'Pine Grove Ave & Irving Park Rd',
 'State St & Kinzie St',
 'Southport Ave & Waveland Ave',
 'Wells St & Erie St',
 'Halsted St & Waveland Ave',
 'State St & 33rd St',
 'Stave St & Armitage Ave',
 'Dearborn Pkwy & Delaware Pl',
 'Franklin St & Lake St',
 'Ashland Ave & Grand Ave',
 'Halsted St & 21st St',
 'State St & 16th St',
 'State St & Van Buren St',
 'Wilton Ave & Diversey Pkwy',
 'Pine Grove Ave & Waveland Ave',
 'Streeter Dr & Illinois St',
 'Clybourn Ave & Division St',
 'Canal St & Adams St',
 'Ellis Ave & 58th St',
 'Clinton St & Madison St',
 'Larrabee St & Webster Ave',
 'Clinton St & Lake St',
 'Shore Drive & 55th St',
 'Lake Shore Dr & North Blvd',
 'Damen Ave & Division St',
 'Clinton St & Roosevelt Rd',
 'Sheffield Ave & Willow St',
 'Lincoln Ave & Fullerton Ave',
 'Green St & Randolph St',
 'Clark St & Schiller St',
 'Martin Luther King Dr & 29th St',
 'Clark St & Elm St',
 'Morgan Ave & 14th Pl',
 'Cityfront Plaza & N Water St',
 'Southport Ave & Belmont Ave',
 'Clifton Ave & Armitage Ave',
 'Wilton Ave & Belmont Ave',
 'Wood St & Taylor St',
 'Halsted St & Willow St',
 'Sedgwick St & Huron St',
 'Wells St & Polk St',
 'Lakeview Ave & Fullerton Pkwy',
 'Michigan Ave & 14th St',
 'McCormick Place',
 'Clark St & Randolph St',
 'State St & Erie St',
 'Noble St & Milwaukee Ave',
 'Dayton St & North Ave',
 'Calumet Ave & 18th St',
 'Hampden Ct & Diversey Pkwy',
 'Morgan St & 31st St',
 'Ashland Ave & Wellington Ave',
 'Logan Blvd & Elston Ave',
 'Sheffield Ave & Addison St',
 'Wolcott Ave & Lawrence Ave',
 'Canal St & Monroe St',
 'Broadway & Argyle St',
 'Sheffield Ave & Wrightwood Ave',
 'Cannon Dr & Fullerton Ave',
 'Leavitt St & Hirsch St',
 'Halsted St & Roscoe St',
 'California Ave & North Ave',
 'Aberdeen St & Jackson Blvd',
 'Woodlawn Ave & 55th St',
 'Eckhart Park',
 'Sedgwick St & North Ave',
 'Lincoln Ave & Diversey Pkwy',
 'Larrabee St & Menomonee St',
 'Morgan St & 18th St',
 'Jefferson St & Monroe St',
 'Milwaukee Ave & Wabansia Ave',
 'Michigan Ave & Oak St',
 'Lake Park Ave & 56th St',
 'Canal St & Jackson Blvd',
 'Marshfield Ave & Cortland St',
 'Damen Ave & Augusta Blvd',
 'Indiana Ave & Roosevelt Rd',
 'Wells St & Walton St',
 'Broadway & Cornelia Ave',
 'Kedzie Ave & Milwaukee Ave',
 'Halsted St & Dickens Ave',
 'Sheridan Rd & Buena Ave',
 'Racine Ave & Fullerton Ave',
 'California Ave & Milwaukee Ave',
 'Lincoln Ave & Waveland Ave',
 'Prairie Ave & Garfield Blvd',
 'Leavitt St & Armitage Ave',
 'Lincoln Ave & Eastwood Ave',
 'Damen Ave & Leland Ave',
 'Halsted St & Polk St',
 'Wood St & Milwaukee Ave',
 'May St & Fulton St',
 'Racine Ave & 18th St',
 'Ashland Ave & Belle Plaine Ave',
 'Damen Ave & Pierce Ave',
 'Ogden Ave & Chicago Ave',
 'Lincoln Ave & Roscoe St',
 'Southport Ave & Irving Park Rd',
 'Carpenter St & Huron St',
 'Blue Island Ave & 18th St',
 'Western Ave & Division St',
 'Wood St & North Ave',
 'Clarendon Ave & Junior Ter',
 'Sangamon St & Washington Blvd',
 'Clarendon Ave & Leland Ave',
 'Southport Ave & Clybourn Ave',
 'Clarendon Ave & Gordon Ter',
 'Fort Dearborn Dr & 31st St',
 'Damen Ave & Cortland St',
 'Montrose Harbor',
 '900 W Harrison',
 'Sheridan Rd & Lawrence Ave',
 'Green St & Milwaukee Ave',
 'Clifton Ave & Lawrence Ave',
 'Damen Ave & Sunnyside Ave',
 'Blackstone Ave & Hyde Park Blvd',
 'Broadway & Wilson Ave',
 'Greenview Ave & Fullerton Ave',
 'Damen Ave & Chicago Ave',
 'Ashland Ave & Lake St',
 'Sedgwick St & Schiller St',
 'Damen Ave & Grand Ave',
 'Paulina St & 18th St',
 'Lincoln Ave & Belle Plaine Ave',
 'Damen Ave & Melrose Ave',
 'Seeley Ave & Roscoe St',
 'Lake Shore Dr & Diversey Pkwy',
 'Sheridan Rd & Montrose Ave',
 'Halsted St & Archer Ave',
 'Franklin St & Quincy St',
 'Lake Shore Dr & Belmont Ave',
 'Southport Ave & Wrightwood Ave',
 'Leavitt St & North Ave',
 'Ashland Ave & Chicago Ave',
 'Damen Ave & Madison St',
 'California Ave & Division St',
 'Greenview Ave & Diversey Pkwy',
 'Michigan Ave & 18th St',
 'Loomis St & Jackson Blvd',
 'Larrabee St & North Ave',
 'State St & Wacker Dr',
 'Lake Shore Dr & Wellington Ave',
 'Damen Ave & Wellington Ave',
 'Ashland Ave & Grace St',
 'Campbell Ave & North Ave',
 'Adler Planetarium',
 'Hermitage Ave & Polk St',
 'Racine Ave & 13th St',
 'Ogden Ave & Congress Pkwy',
 'Racine Ave & 15th St',
 'Paulina St & Montrose Ave',
 'Clinton St & 18th St',
 'Orleans St & Elm St',
 'State St & 29th St',
 'Ravenswood Ave & Irving Park Rd',
 'Western Ave & Leland Ave',
 'Ravenswood Ave & Berteau Ave',
 'Ashland Ave & 21st St',
 'Racine Ave & Congress Pkwy',
 'Ogden Ave & Race Ave',
 'Ada St & Washington Blvd',
 'Kimbark Ave & 53rd St',
 'Milwaukee Ave & Rockwell St',
 'Clark St & Montrose Ave',
 'Ashland Ave & Blackhawk St',
 'Kedzie Ave & Palmer Ct',
 'Lake Shore Dr & Monroe St',
 'Claremont Ave & Hirsch St',
 'Wells St & 19th St',
 'Racine Ave & Wrightwood Ave',
 'Calumet Ave & 33rd St',
 'Wood St & Grand Ave',
 'Halsted St & 35th St',
 'Rhodes Ave & 32nd St',
 'Clark St & Leland Ave',
 'Emerald Ave & 31st St',
 'Emerald Ave & 28th St',
 'Damen Ave & Cullerton St',
 'State St & 35th St',
 'Ashland Ave & Wrightwood Ave',
 'Lincoln Ave & Addison St',
 'California Ave & Francis Pl',
 'Indiana Ave & 26th St',
 'King Dr & 47th St',
 'May St & Cullerton St',
 'Southport Ave & Clark St',
 'California Ave & 21st St',
 'Wallace Ave & 35th St',
 'Normal Ave & Archer Ave',
 'Indiana Ave & 40th St',
 'Greenwood Ave & 47th St',
 'Indiana Ave & 31st St',
 'Paulina St & Diversey Pkwy',
 'Cottage Grove Ave & 51st St',
 'Cottage Grove Ave & Oakwood Blvd',
 'Western Ave & 21st St',
 'Western Ave & 24th St',
 'Ashland Ave & 13th St',
 'Halsted St & 37th St',
 'Cottage Grove Ave & 47th St',
 'Martin Luther King Dr & Oakwood Blvd',
 'Calumet Ave & 35th St',
 'Cottage Grove Ave & 43rd St',
 'Damen Ave & Coulter St',
 'Leavitt St & Lawrence Ave',
 'Clark St & Grace St',
 'Union Ave & Grand Ave',
 'Cityfront Plaza Dr & Pioneer Ct',
 'Ravenswood Ave & Lawrence Ave',
 'Wabash Ave & Wacker Pl',
 'Racine Ave & 19th St',
 'MLK Jr Dr & 29th St',
 'Damen Ave & Clybourn Ave',
 'MLK Jr Dr & 47th St',
 'MLK Jr Dr & Oakwood Blvd']

In [7]:
removed14 = list(set(allSt14) - set(allSt15))
len(removed14)

16

In [8]:
added15 = list(set(allSt15) - set(allSt14))
len(added15)

196

# Plotting all added stations of 2015 and removed stations of 2014

In [9]:
#reading the 'final_stations.csv' into a dataframe
#using all the stations co-ordinates and comparing with each of the year's stations added/ removed list and adding locations 
#     to each of the list to plot yearly data.

final = pd.read_csv('final_stations.csv')

In [10]:
final.head(1)

,Unnamed: 0,Station,Latitude,Longitude
0,0,Ogden Ave & Race Ave,41.891795,-87.658751


In [11]:
#Removing the unnecessary column in the dataframe
final = final.drop('Unnamed: 0', axis =1)

In [12]:
final.head(1)

,Station,Latitude,Longitude
0,Ogden Ave & Race Ave,41.891795,-87.658751


In [13]:
#Converting the dataframe into a dictionary
# key : Station name and value: latitude and longitude

locations = dict()

for station in final['Station']:
    locations[station] = {'lat': final[final['Station'] == station]['Latitude'].to_list()[0], 'long': final[final['Station'] == station]['Longitude'].to_list()[0]}

#printing out the first item in the dictionary to check the key-value pair
list(locations.items())[0]

('Ogden Ave & Race Ave', {'lat': 41.891795, 'long': -87.658751})

In [14]:
#Adding co-ordinates to all the stations that are added this year
n15 = {k: locations[k] for k in locations.keys() & set(added15)}

#printing out the first item in the dictionary to check the key-value pair
list(n15.items())[0]

('Ridge Blvd & Touhy Ave', {'lat': 42.012132277, 'long': -87.682912644})

In [15]:
#Converting the dictionary into dataframe
st15 = pd.DataFrame.from_dict(n15, orient='index')

In [16]:
#Resetting the index
st15.reset_index(inplace=True)

#Renaming the index column to Station name
st15 = st15.rename(columns = {'index':'Station Name'})

In [17]:
#Adding a column with values of color for every station
st15['color'] = st15.apply(lambda row: 'green', axis=1)

#printing to check the structure of the dataframe
st15.head(1)

,Station Name,lat,long,color
0,Ridge Blvd & Touhy Ave,42.012132,-87.682913,green


In [18]:
removed14 = ['State St & Erie St',
 'Martin Luther King Dr & 29th St',
 'Lincoln Ave & Eastwood Ave',
 'King Dr & 47th St',
 'Wallace Ave & 35th St',
 'Racine Ave & 19th St',
 'Wolcott Ave & Lawrence Ave',
 '900 W Harrison',
 'Halsted St & Waveland Ave',
 'Green St & Milwaukee Ave',
 'Cityfront Plaza & N Water St',
 'Martin Luther King Dr & Oakwood Blvd',
 'Paulina St & Diversey Pkwy',
 'Clark St & Waveland Ave',
 'St Clair St & Erie St',
 'State St & Wacker Dr']

In [19]:
#adding co-ordinates for the removed stations in 2014
rm14 = {k: locations[k] for k in locations.keys() & set(removed14)}

#printing out the first item in the dictionary to check the key-value pair
list(rm14.items())[0]

('State St & Erie St', {'lat': 41.893975, 'long': -87.628623})

In [20]:
#Converting the dictionary into dataframe
rm14 = pd.DataFrame.from_dict(rm14, orient='index')

In [21]:
#Resetting the index
rm14.reset_index(inplace=True)

#Renaming the index column to Station name
rm14 = rm14.rename(columns = {'index':'Station Name'})

In [22]:
#Adding a column with values of color for every station
rm14['color'] = rm14.apply(lambda row: 'orange', axis=1)

#printing to check the structure of the dataframe
rm14.head(1)

,Station Name,lat,long,color
0,State St & Erie St,41.893975,-87.628623,orange


# Location Marker map for interactivity purpose (Station Name)

In [23]:
#merging both the added15 and removed14 dataframes
merged = pd.concat([rm14, st15], axis =0)

In [24]:
#setting marker as the color mentioned in the color column of the dataframe
marker = merged.loc[merged['color']!='']

map = folium.Map(location=[marker.lat.mean(), marker.long.mean()], zoom_start=12,
                     width=980,height=590, control_scale=True, tiles='OpenStreetMap')
marker.apply(lambda row:folium.Marker(location=[row["lat"], row["long"]],
        popup=row['Station Name'],
          icon=folium.Icon(color=row['color'])
            ).add_to(map), axis=1)

map

# Circle marker map 2015
## Saving the map for converting into gif

In [25]:
#create a map
m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='cartodb positron')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2015')   
m.get_root().html.add_child(folium.Element(title_html))

#creates a CircleMarker and adds it to the map
def plotDot(point):
    folium.CircleMarker(location=[point.lat, point.long],             
                        color= "green",
                        radius=3,
                        weight=2
                       ).add_to(m)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
st15.apply(plotDot, axis = 1)

m

In [26]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [27]:
m.save('Gif_circlemarker/2015.html')

In [28]:
with fileinput.FileInput('Gif_circlemarker/2015.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [29]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_circlemarker')

fn='2015.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2015.png')
browser.quit()

In [30]:
image = Image.open('ss2015.png')
box = (0, 0, 1100, 700)
cropped_image = image.crop(box)
cropped_image.save('croppedss2015.png')

# Heatmap 2015
## Saving the map for converting into gif

In [31]:
numlist = st15.reset_index()[['lat', 'long']].values.tolist()


m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='OpenStreetMap')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2015')   
m.get_root().html.add_child(folium.Element(title_html))


HeatMap(numlist).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

m

In [32]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [33]:
m.save('Gif_heatmap/2015.html')

In [34]:
with fileinput.FileInput('Gif_heatmap/2015.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [35]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_heatmap')

fn='2015.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2015.png')
browser.quit()

In [36]:
image = Image.open('ss2015.png')
box = (0, 0, 1250, 780)
cropped_image = image.crop(box)
cropped_image.save('croppedss2015.png')